<a href="https://colab.research.google.com/github/ParthikB/Neural-Networks-from-Scratch-2/blob/NN-v2.0/Neural%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Helper Functions


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

################## helper functions ######################

# Generating a basic dataset.
def create_data(total_samples, range_of_data):
    X1, X2, Y = [], [], []

    for datapoints in range(total_samples):
        x2 = np.random.randint(1, range_of_data + 1)
        x1 = np.random.randint(1, range_of_data + 1)

        if x1 < range_of_data / 2:
            label = 0
        else:
            label = 1
        X1.append(x1)
        X2.append(x2)
        X = np.array([X1, X2])
        Y.append(label)

    return np.array(X).reshape(2, -1), np.array(Y).reshape(1, -1)

# Defining Activation Functions
def sigmoid(z):
  A =  1 / (1 + np.exp(-z))
  cache = z
  return A, cache


def relu(Z):
    A = np.maximum(0, Z)
    cache = Z
    return A, cache

def relu_derivative(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0

    return dZ

def sigmoid_derivative(dA, cache):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ


def initialize_random_parameters(layer_dims, X):
    parameters = {}
    parameters["W1"] = np.random.randn(layer_dims[0], X.shape[0]) * 0.01
    parameters["b1"] = np.zeros((layer_dims[0], 1))
    for i in range(1, len(layer_dims)):
        parameters["W" + str(i+1)] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        parameters["b" + str(i+1)] = np.zeros((layer_dims[i], 1))

    return parameters
  
# To check the accuracy of our Model.
def accuracy_score(Yhat, Y):
  Yhat = np.where(Yhat < 0.5, 0, 1)
  accuracy = 100 - np.mean(np.abs(Yhat-Y) * 100)
  return accuracy

# To visualize our dataset/predictions.
def plot_data(data, type_of_data):
    X = data[0]
    Y = data[1]
    sns.scatterplot(X[0], X[1], hue=Y[0])
    plt.xlabel('Feature 1')
    plt.ylabel("Feature 2")
    plt.title(type_of_data)
    type_of_data = 'Plots : ' + type_of_data + ".png"
    plt.savefig(type_of_data)
    plt.close()

    
def plot_cost_function(epoch_log, cost_log):
  plt.plot(epoch_log, cost_log)
  plt.xlabel('Epochs')
  plt.ylabel("Cost")
  plt.title("Cost Function")
  plt.savefig('Plots : Cost Function.png')
  plt.close()

### Basic Neuron Working

In [0]:
# Defining main Class.
class Neuron:
  
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias   = bias
    
    
  def feedforward(self, X):
    # Weights inputs, add bias and then use activation function
    math_e_magic = np.dot(self.weights, X) + self.bias
    output, _ = sigmoid(math_e_magic)
    
    return output
    
    
# Testing
X = [2, 3]
weights, bias = [2, 3], [4]

# > Defining the Class
neuron = Neuron(weights, bias)

# > Feedforwarding
print(neuron.feedforward(X))
# Result --> array([0.99999996])

[0.99999996]


### Neural Network Class

In [0]:
# from helpers import *

class NeuralNetwork:    
  
  # Defining the Feedforward function
  def feedforward(self, X, parameters, activation_used):

    def linear_forward(A, W, b, activation):
      Z = np.dot(W, A) + b
      linear_cache = (A, W, b)

      if activation == 'sigmoid':
        A, activation_cache = sigmoid(Z)
      elif activation == 'relu':
        A, activation_cache = relu(Z)
      
      # Saving some variables that will be needed later in Back Propagation in the form of caches. You don't need to understand them for now.
      cache = (linear_cache, activation_cache)
      return A, cache

    caches = []
    A = X
    L = len(parameters) // 2  # Total number of Layers in our Network
    
    # Iterating over every Layer and computing the Activations.
    # See how different activations are being used.
    # You can play with the activations in the hidden layers, but the Output layer activation is always set to 'sigmoid'.
    for i in range(1, L):
        A, cache = linear_forward(A, parameters["W" + str(i)], parameters["b" + str(i)], activation_used)
        caches.append(cache)

    A, cache = linear_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], 'sigmoid')
    caches.append(cache)

    return A, caches

  # Defining the cost function.
  def cost(self, yhat, y):
    m = y.shape[1]
    cost = -np.sum(y * np.log(yhat) + (1-y) * np.log(1-yhat)) / m
    return cost
  
  # Defining the Back Progation Algorithm. You can skip this part for now.
  def backward_propagation(self, yhat, y, caches, activation_used):

    def linear_backward(dA, cache, activation):
      linear_cache, activation_cache = cache
      A, W, b = linear_cache

      if activation == 'sigmoid':
        dZ = sigmoid_derivative(dA, activation_cache)
      elif activation == 'relu':
        dZ = relu_derivative(dA, activation_cache)  
        
      A_prev = A
      m = A_prev.shape[1]

      dW = np.dot(dZ, A_prev.T) / m
      db = np.sum(dZ, axis=1, keepdims=True) / m
      dA_prev = np.dot(W.T, dZ)

      return dA_prev, dW, db


    L = len(caches)
    grads = {}
    y = y.reshape(yhat.shape)
    dyhat = -(np.divide(y, yhat) - np.divide(1-y, 1-yhat))
    dAL = dyhat
    
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_backward(dAL, current_cache, 'sigmoid')
    for l in range(L-1)[::-1]:
        current_cache = caches[l]
        grads["dA" + str(l)], grads["dW" + str(l+1)], grads["db" + str(l+1)] = linear_backward(grads["dA" + str(l+1)], current_cache, activation_used)

    return grads
  
  # Defining the Gradient Descent Algorithm. Learning Rate is set to default at 0.01
  def gradient_descent(self, parameters, grads, learning_rate=0.01):
    L = parameters.__len__() // 2
    
    for l in range(1, L+1):
      parameters["W" + str(l)] -= learning_rate * grads["dW" + str(l)]
      parameters["b" + str(l)] -= learning_rate * grads["db" + str(l)]

    return parameters

### Training and Testing

In [0]:
'''
  Welcome to my Playground! Here, you can run the Neural Network and even tinker with the parameters to discover new combinations and reach higher accuracies!

  IMPORTANT NOTE : 
    > If you want to change the parameters, you'll first need to signup and then fork this repl.
      I can't help it, it's in 'repl.it terms'.
    > You don't need to change anything else. But you can if you wish to.
    > The output visuals will be automatically saved in the Files Tab on the left of the screen. You      can access them from there.

  Have fun!
'''

# # Importing helper functions
# from network_functions import *
# from helpers import *
# import matplotlib as mpl

# CHANGABLE PARAMETERS
TRAINING_SAMPLES = 1000 # Total number of training samples.
LAYER_DIMS = [2, 1]     # Note that the input Layer is predefined so you don't need to define it again.
EPOCHS = 2500           # Total number of Iterations.
LEARNING_RATE = 0.04    # Learning Rate to be used in Gradiend Descent.
ACTIVATION = 'relu'     # Activations used in Neural Network. Try jumping between relu/sigmoid

print(f'''
Training Samples : {TRAINING_SAMPLES}
Layer Dimensions : {LAYER_DIMS}
Epochs           : {EPOCHS}
Learning Rate    : {LEARNING_RATE}
''')

# Creating Data
X, y = create_data(TRAINING_SAMPLES, 100)
plot_data([X, y], 'Dataset')

# Initializing Random Parameters
parameters = initialize_random_parameters(LAYER_DIMS, X)

# Few logs just to keep track of our training.
cost_log, epoch_log = [], []

# Creating the Network
nn = NeuralNetwork()

# Training
print("Initializing Training...")
for epoch in range(EPOCHS):
  
  if epoch % 100 == 0 and epoch != 0:
    LEARNING_RATE -= LEARNING_RATE/10     # This is called Learning Rate Decay. It is basically done to optimize our Training.
    print("Epoch :", epoch)
  
  # Feedforwarding
  yhat, caches = nn.feedforward(X, parameters, ACTIVATION)
  # Computing and saving the logs for plotting
  cost = nn.cost(yhat, y)
  cost_log.append(cost)
  epoch_log.append(epoch+1)
  
  # Back Propagation
  grads = nn.backward_propagation(yhat, y, caches, ACTIVATION)
  # Gradient Descent
  parameters = nn.gradient_descent(parameters, grads, LEARNING_RATE)
  
  
predictions = yhat  # yhat --> the predicted output

print()
print("********** Accuracy :", accuracy_score(predictions, y), "% **********")
print("// Graphs saved. Check the files tab.")

# Saving the Cost Function Graph
plot_cost_function(epoch_log, cost_log)

# Just another way to convert predictions to 0s and 1s
yhat = np.where(predictions<0.5, 0, 1)

# Saving our Predictions graph
plot_data([X, yhat], 'Prediction')


'''
  Awesome! You just created your First Neural Network from Scratch!

  NOTE:
    > You might not be getting an amazing accuracy. That's because there are various things that we've skipped and various parameters that we haven't optimized just to not go beyond the scope of this article.

  Though, you can try to tinker with the 3 parameters:
    > TRAINING_SAMPLES
    > LAYER_DIMS
    > EPOCHS
    > LEARNING_RATE

  Lemmi hear your adventures and accuracies through your comments!
  Peace out.

'''


Training Samples : 1000
Layer Dimensions : [2, 1]
Epochs           : 2500
Learning Rate    : 0.04

Initializing Training...
Epoch : 100
Epoch : 200
Epoch : 300
Epoch : 400
Epoch : 500
Epoch : 600
Epoch : 700
Epoch : 800
Epoch : 900
Epoch : 1000
Epoch : 1100
Epoch : 1200
Epoch : 1300
Epoch : 1400
Epoch : 1500
Epoch : 1600
Epoch : 1700
Epoch : 1800
Epoch : 1900
Epoch : 2000
Epoch : 2100
Epoch : 2200
Epoch : 2300
Epoch : 2400

********** Accuracy : 78.0 % **********
// Graphs saved. Check the files tab.


"\n  Awesome! You just created your First Neural Network from Scratch!\n\n  NOTE:\n    > You might not be getting an amazing accuracy. That's because there are various things that we've skipped and various parameters that we haven't optimized just to not go beyond the scope of this article.\n\n  Though, you can try to tinker with the 3 parameters:\n    > TRAINING_SAMPLES\n    > LAYER_DIMS\n    > EPOCHS\n    > LEARNING_RATE\n\n  Lemmi hear your adventures and accuracies through your comments!\n  Peace out.\n\n"